In [1]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")
import json
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer

In [2]:
import re
import operator
fileList=['GPT']
resultDf=pd.DataFrame()
for num,fileName in enumerate(fileList):
    print(fileName)
    dfs=pd.read_excel("FinalDatasetAuto"+fileName+".xlsx",index_col=0)
    dfs4=pd.read_excel("Label.xlsx")
    dfs4=dfs4[427:].reset_index(drop=True)
    dfs=dfs[427:]
    dfs=dfs.reset_index(drop=True)
    for index,i in enumerate(dfs['adj/noun']):
        if dfs.loc[index,'adj/noun-label']=='N':
            dfs.loc[index,'arg0']=''
            dfs.loc[index,'arg1']=''
            dfs.loc[index,'V']=''
            dfs.loc[index,'baseV']=''
            dfs.loc[index,'PP']=''
    dfs['arg0']=dfs['arg0'].replace(np.NaN,'')
    dfs['arg1']=dfs['arg1'].replace(np.NaN,'')
    dfs['PP']=dfs['PP'].replace(np.NaN,'')
    findIt=False
    columnList=['arg0','baseV','arg1','PP']
    for index,i in enumerate(dfs['adj/noun']):
        result=''
        for column in columnList:
            if dfs.loc[index,column]!='':
                    result=result+dfs.loc[index,column]+" "
        dfs.loc[index,'gold']=result.strip()
    for index,i in enumerate(dfs['adj/noun']):
        if index+1<len(dfs['adj/noun']) and dfs.loc[index,'adj/noun']==dfs.loc[index+1,'adj/noun'] and dfs.loc[index,'n_v']==dfs.loc[index+1,'n_v'] and dfs.loc[index,'prep']==dfs.loc[index+1,'prep'] and dfs.loc[index,'pobj']==dfs.loc[index+1,'pobj']:
            dfs.loc[index,'gold2']=dfs.loc[index+1,'gold']
        else:
            dfs.loc[index,'gold2']=dfs.loc[index,'gold']
    for index,i in enumerate(dfs['gold']):
        if not pd.isnull(i):
            doc=nlp(i)
            reducedGold=''
            for word in doc:
                if word.tag_!='DT':
                        reducedGold=reducedGold+word.lemma_+" "
            dfs.loc[index,'reducedGold']=reducedGold.lower()
        reducedGold3=''
        for para in dfs.loc[index,'gold'].split():
            doc=nlp(para)
            for word in doc:
                if word.tag_!='DT':
                    if word.text!='a' and word.text!='an' and word.text!='the':
                        answer=lemmatizer(word.text, NOUN)
                        reducedGold3=reducedGold3+answer[0]+" "
        dfs.loc[index,'reducedGold3']=reducedGold3.lower()
    for index,i in enumerate(dfs['gold2']):
        if not pd.isnull(i):
            doc=nlp(i)
            reducedGold=''
            for word in doc:
                if word.tag_!='DT':
                    reducedGold=reducedGold+word.lemma_+" "
            dfs.loc[index,'reducedGold2']=reducedGold.lower()
        reducedGold4=''
        for para in dfs.loc[index,'gold2'].split():
            doc=nlp(para)
            for word in doc:
                if word.tag_!='DT':
                    if word.text!='a' and word.text!='an' and word.text!='the':
                        answer=lemmatizer(word.text, NOUN)
                        reducedGold4=reducedGold4+answer[0]+" "
        dfs.loc[index,'reducedGold4']=reducedGold4.lower()
    dfs2=pd.read_excel("FinalDatasetAutoTextualEntailmentTop3RemoveArticleMetric.xlsx")
    dfs3=pd.read_excel("FinalDatasetAMR.xlsx",index_col=0)
    dfs['prediction']=dfs3['prediction']
    dfs['HighestScorePatternMlist']=dfs2['HighestScorePatternMlist']
    dfs['HighestScorePatternNotMlist']=dfs2['HighestScorePatternNotMlist']
    for index,i in enumerate(dfs2['adj/noun']):
        if not pd.isnull(dfs.loc[index,'v']):
            if dfs.loc[index,'prediction']=='M':
                scoreDict={}
                for pattern in dfs.loc[index,'HighestScorePatternMlist'].split(","):
                    scoreDict[pattern]=dfs.loc[index,pattern]
                dfs.loc[index,'HighestScorePattern']=max(scoreDict.items(), key=operator.itemgetter(1))[0][:-5]
            if dfs.loc[index,'prediction']=='NotM':
                scoreDict={}
                for pattern in dfs.loc[index,'HighestScorePatternNotMlist'].split(","):
                    scoreDict[pattern]=dfs.loc[index,pattern]
                dfs.loc[index,'HighestScorePattern']=max(scoreDict.items(), key=operator.itemgetter(1))[0][:-5]
            if dfs.loc[index,'HighestScorePattern']=='BVO':
                dfs.loc[index,'HighestScorePattern']='VOB'
            if dfs.loc[index,'HighestScorePattern']=='OBV':
                dfs.loc[index,'HighestScorePattern']='OVB'
    for index,i in enumerate(dfs['adj/noun']):
        #print(index)
        dfs.loc[index,'OriginalHighestScorePattern']=dfs.loc[index,'HighestScorePattern']
        if dfs.loc[index,'HighestScorePattern']=='MVpOPassive':
            pattern=dfs.loc[index,'HighestScorePattern']
            if "by" in dfs.loc[index,pattern].split():
                dfs.loc[index,'HighestScorePattern']='OVM'
            else:  
                dfs.loc[index,'HighestScorePattern']='MVpOPassiveVMpO'
                dfs.loc[index,'HighestScorePattern']='VMpO'
        if dfs.loc[index,'HighestScorePattern']=='MpOVPassive':
                dfs.loc[index,'HighestScorePattern']='MpOVPassiveVMpO'
                dfs.loc[index,'HighestScorePattern']='VMpO'
        if dfs.loc[index,'HighestScorePattern']=='OVpMPassive':
            pattern=dfs.loc[index,'HighestScorePattern']
            if "by" in dfs.loc[index,pattern].split():
                dfs.loc[index,'HighestScorePattern']='MVO'
            else:
                dfs.loc[index,'HighestScorePattern']='OVpMPassiveVOpM'
                dfs.loc[index,'HighestScorePattern']='VOpM'
    remove_lower = lambda text: re.sub('[a-z]','',text)
    for index,i in enumerate(dfs['adj/noun']):
        if not pd.isnull(dfs.loc[index,'HighestScorePattern']):
            #print(dfs.loc[index,'HighestScorePattern'],remove_lower(dfs.loc[index,'HighestScorePattern']))
            dfs.loc[index,'HighestScorePattern']=remove_lower(dfs.loc[index,'HighestScorePattern'])
            #print(dfs.loc[index,'HighestScorePattern'],)
    for index,i in enumerate(dfs['adj/noun']):
        if pd.isnull(dfs.loc[index,'v']) or dfs.loc[index,'HighestScorePattern']=='M' or dfs.loc[index,'HighestScorePattern']=='V1' or dfs.loc[index,'HighestScorePattern']=='V2' or dfs.loc[index,'HighestScorePattern']=='OVB' or dfs.loc[index,'HighestScorePattern']=='VOB' or dfs.loc[index,'HighestScorePattern']=='BVO' or dfs.loc[index,'HighestScorePattern']=='OBV':
            dfs.loc[index,'HighestScorePattern']='OvMN/vMNO'
    for index,i in enumerate(dfs['adj/noun']):
        dfs.loc[index,'goldAnswerType']=dfs4.loc[index,'goldAnswerType']
        correct=False
        for prediction in dfs.loc[index,'HighestScorePattern'].split("/"):
            if prediction in dfs.loc[index,'goldAnswerType']:
                correct=True
        if correct==True:
            dfs.loc[index,'FinalLabel']='True'
        else:
            dfs.loc[index,'FinalLabel']='False'
    dfs=dfs.fillna('')
    for index,i in enumerate(dfs['adj/noun']):
        for columns in dfs.columns:
            dfs.loc[index,columns]=str(dfs.loc[index,columns])
    dfs=dfs.groupby(['adj/noun','n_v','prep','pobj'],sort=False)[dfs.columns[4:]].agg('//'.join).reset_index()
    dfs['final']=dfs['FinalLabel']
    for index,i in enumerate(dfs['adj/noun']):
        if 'True' in dfs.loc[index,'FinalLabel'].split("//"):
            dfs.loc[index,'FinalLabel']='True'
        else:
            dfs.loc[index,'FinalLabel']='False'
    print(dfs['FinalLabel'].value_counts())
    print("Accuracy= "+str(dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel'])))
    resultDf.loc[num,'model']=fileName
    resultDf.loc[num,'True']=dfs['FinalLabel'].value_counts()['True']
    resultDf.loc[num,'False']=dfs['FinalLabel'].value_counts()['False']
    resultDf.loc[num,'Accuracy']=str(dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel']))
    dfs.to_excel("FinalDatasetAuto"+fileName+"Top3BasedTextualEntailmentWordOrderMetric.xlsx")

GPT
False    94
True     80
Name: FinalLabel, dtype: int64
Accuracy= 0.45977011494252873
